In [1]:
%pip install spacy pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pymupdf  # PyMuPDF
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    text = ""
    with pymupdf.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def process_documents_in_folder(folder_path):
    processed_docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)
            entities = extract_entities(text)
            processed_docs.append((pdf_path, entities))
    return processed_docs


In [3]:
%pip install py2neo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from neo4j import GraphDatabase, basic_auth, Config
import ssl
import logging
uri = "bolt://c24a04a8.databases.neo4j.io:7687"
username = "neo4j"
password = "TIEXesTzQ3V72Od9m-64Kpvd6AxVw8uB4fyvYODD_Jk"

# Create a custom SSL context that skips verification
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Connect to Neo4j using the custom SSL context
driver = GraphDatabase.driver(uri, auth=basic_auth(username, password), encrypted=True, ssl_context=ssl_context)

def populate_knowledge_graph(docs):
    with driver.session() as session:
        for pdf_path, entities in docs:
            doc_id = pdf_path.split("/")[-1]  # Use file name as document ID
            doc_type = "Invoice" if "invoice" in pdf_path.lower() else "Contract"
            
            for entity, label in entities:
                session.run(
                    "MERGE (n:Entity {name: $name, document_type: $document_type, document_id: $document_id})",
                    name=entity, document_type=doc_type, document_id=doc_id
                )

folder_path = "uploads"
processed_docs = process_documents_in_folder(folder_path)
populate_knowledge_graph(processed_docs)


C:\Users\Abhay\AppData\Local\Temp\ipykernel_22320\3983335161.py:1: DeprecationWarning: Importing Config from neo4j is deprecated without replacement. It's internal and will be removed in a future version.
  from neo4j import GraphDatabase, basic_auth, Config


In [5]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")

def embed_entities(docs):
    embeddings = {}
    for _, entities in docs:
        for entity, _ in entities:
            if entity not in embeddings:
                embeddings[entity] = embedding_model.embed(entity)
    return embeddings

entity_embeddings = embed_entities(processed_docs)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
import llm_calls as lc

def retrieve_information(query_text):
    with driver.session() as session:
        result = session.run(
            "MATCH (n) WHERE n.name CONTAINS $query_text RETURN n.name AS name", 
            query_text=query_text
        )
        return [record["name"] for record in result]




query = "Tell me about Michael Roberts"
retrieved_info = retrieve_information(query)
print(retrieved_info)
print("=="*20)
augmented_input = f"Query: {query}\nKnowledge Graph Info: {retrieved_info}"
response = lc.llm_generate(augmented_input)
print(response)


<function retrieve_information at 0x000001AB9908E3E0>
Michael Roberts is an American educator, author, and expert in the field of educational technology. He is the Chief Academic Officer at McGraw-Hill Education and has made significant contributions to the development of educational software and online learning platforms.

Early Life and Education:

Michael Roberts received his Bachelor's degree in Education from the University of Nebraska at Omaha and his Master's degree in Educational Technology from the University of Nebraska at Lincoln.

Career:

Michael began his career in education as a high school teacher in Omaha, Nebraska. He then joined the University of Nebraska at Omaha as an instructor in the Department of Education. In 2001, he left academia to join the McGraw-Hill Education team, where he has held various leadership roles, including Assistant Vice President, Product Management and Development.

Recognition:

Michael has received numerous awards and recognitions for his 